#### **Import Libraries and Configuration**

In [1]:
# Import required functions and classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched, predict 
from sahi.prediction import visualize_object_predictions
from IPython.display import Image
from numpy import asarray
import cv2
import os
import time
from PIL import Image
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult
from pathlib import Path
from multiprocessing import Pool, cpu_count
import torch
from torchvision.ops import nms, clip_boxes_to_image

/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download YOLOv8-S model to 'models/yolov8s.pt'
yolov8_model_path = 'models/yolov8/last.pt'
#download_yolov8s_model(destination_path=yolov8_model_path)

In [3]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [4]:
def get_slice_parameters(object_density, slice_size):
    
    #start_time = time.time()
    #image_path = "test_data/0000006_06773_d_0000018.jpg"
    #image = Image.open(image_path).convert("RGB")
    #image_width, image_height  = image.size
    #print("Image Width:", image_width)
    #print("Image Height:", image_height)
    #min_dim = min(image_width, image_height)
    #slice_size = min_dim // 4 if min_dim > 1600 else min_dim // 2
    #print(f"Dimension calculation time taken: {(time.time() - start_time)*1000:.2f} ms")

    
    if object_density >= 50:
        #slice_size = min_dim // 4
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        return

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [5]:
def nms_merge(predictions, iou_threshold=0.5):
    if not predictions:
        return []
    boxes = torch.tensor([p.bbox.to_xyxy() for p in predictions])
    scores = torch.tensor([p.score.value for p in predictions])
    keep_indices = nms(boxes, scores, iou_threshold).tolist()
    #keep_indices = clip_boxes_to_image(boxes, 32).tolist()
    return [predictions[i] for i in keep_indices]

In [6]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

In [7]:
def merged_preds_to_coco(preds, image_id):
    coco_predictions = []
    for pred in preds:
        try:
            coco = pred.to_coco_prediction(image_id=image_id)
            coco_dict = {
                "image_id": coco.image_id,
                "bbox": coco.bbox,
                "score": coco.score,
                "category_id": coco.category_id,
                "segmentation": coco.segmentation,
                "iscrowd": coco.iscrowd,
                "area": coco.area,
            }
            # Optionally include category_name (not used in COCO eval)
            # coco_dict["category_name"] = coco.category_name

            # Only add if bbox is valid (non-empty)
            if coco_dict["bbox"]:
                coco_predictions.append(coco_dict)
        except Exception as e:
            print(f"Failed to convert to COCO dict: {e}")
    return coco_predictions


In [20]:
def predict_fine_sliced_images_refactored_2(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\nRunning fine slicing prediction on {len(image_files)} image(s)...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)

        img_id = get_image_id(data, filename_wo_ext) if dataset_json_path else None
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                object_density = len(base_pred.object_prediction_list)
                print("Object Density:", object_density)

                slice_params = get_slice_parameters(object_density, base_slice_size)

                iteration_time = time_end

                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="NMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    preds = sliced_pred.object_prediction_list
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    preds = base_pred.object_prediction_list

                # Offset bounding boxes back to original image space
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)

                total_prediction_count += len(preds)
                total_prediction_time += iteration_time

        print("________________________________________________")

        # Merge all predictions for image
        merged_preds = nms_merge(all_object_predictions, iou_threshold=0.5)
        
        # Convert merged predictions to COCO format
        if dataset_json_path:
            coco_preds = merged_preds_to_coco(merged_preds, img_id)
            all_coco_preds.extend(coco_preds)
            
        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=merged_preds,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\nCompleted {len(image_files)} images.")
    print("Total Prediction Count (Before NMS Merge): ", grand_total_predictions)
    print("Total Prediction Count (After Final NMS Merge): ", len(all_coco_preds))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


#### **Predictions & Evaluation (Single Image)**

In [108]:
#latest - OptNMS Take6
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000006_06773_d_0000018
Cropped Image: 0 0 680 382
Initial Prediction time is: 8.59 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 61.59 ms
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.39 ms
Object Density: 6
Prediction time is: 7.39 ms
Cropped Image: 0 382 680 764
Initial Prediction time is: 7.37 ms
Obje

In [109]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp320/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.422
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.754
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.493
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.976
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.750
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.479
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [100]:
#latest - OptNMS Take6
source_folder = './single_test/images_bk'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000006_06773_d_0000018
Cropped Image: 0 0 680 382
Initial Prediction time is: 20.31 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 61.93 ms
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.36 ms
Object Density: 6
Prediction time is: 7.36 ms
Cropped Image: 0 382 680 764
Initial Prediction time is: 7.32 ms
Obj

In [101]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp317/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.694
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.434
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.976
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.664
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.424
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [110]:
#Threshold IOU = 0.5
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_visdrone_test_990.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000006_06773_d_0000018
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]

Original Prediction Count 143
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Prediction time is: 491.99 ms
Prediction results are successfully exported to runs/predict/exp235
Model loaded in 0.032637596130371094 seconds.
Slicing performed in 0.0013730525970458984 seconds.
Prediction performed in 0.4919910430908203 seconds.
Exporting performed in 0.049646854400634766 seconds.


In [111]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './runs/predict/exp235/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.704
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.958
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.741
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [102]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_visdrone_test_990.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000006_06773_d_0000018
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]

Original Prediction Count 143
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Prediction time is: 488.65 ms
Prediction results are successfully exported to runs/predict/exp234
Model loaded in 0.10470438003540039 seconds.
Slicing performed in 0.0013213157653808594 seconds.
Prediction performed in 0.4886488914489746 seconds.
Exporting performed in 0.0502934455871582 seconds.


In [103]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './runs/predict/exp234/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.704
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.958
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.741
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [97]:
#latest - OptNMS Take6
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Initial Prediction time is: 9.78 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 61.60 ms
Cropped Image: 680 0 1360 382
Initial Pre

In [98]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp316/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.044
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [95]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp315/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.044
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp272/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.019
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [91]:
#latest - OptNMS Take5
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Initial Prediction time is: 8.67 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Original Prediction Count 32
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 

In [92]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp270/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.019
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [36]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

Original Prediction Count 271
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  11
Prediction time is: 510.31 ms
Prediction results are successfully exported to runs/predict/exp233
Model loaded in 0.18946099281311035 seconds.
Slicing performed in 0.001336812973022461 seconds.
Prediction performed in 0.5103139877319336 seconds.
Exporting performed in 0.04958391189575195 seconds.


In [37]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp233/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.038
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [23]:
#latest - OptNMS Take4
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Initial Prediction time is: 8.60 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Original Prediction Count 32
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 

In [15]:
#latest - OptNMS Take3
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Original Prediction Count 32
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Level Prediction Count 1:  25
Cropped Image: 680 0 1360 

In [14]:
#latest - OptNMS Take2
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  OptNMS
Original Prediction Count 37
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  32
Final Bounding Box Count (OptNMS): 20
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Level Prediction Count 1:  24
Cropped Image: 680 0 1360 382
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height

In [10]:
#latest - NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  NMS
Original Prediction Count 37
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Level Prediction Count:  19
Total Prediction Count:  19
Cropped Image: 680 0 1360 382
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  NMS
Original Prediction Count 41
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS): 

In [43]:
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)



🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  OptNMS
Original Prediction Count 37
Intial Min Area:  32
Confidence Scores:  0.3
Min area Threshold:  32
Adaptive Filtered Prediction:  32
Final Bounding Box Count (OptNMS): 20
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Level Prediction Count:  24
Total Prediction Count:  24
Cropped Image: 680 0 1360 382
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width R

#### **Prediction & Evaluation (10 Images)**

In [11]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 10.32 ms
Object Density: 0
Prediction time is: 10.32 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.05 ms
Object Density: 0
Prediction time is: 9.05 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.94 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  242
Final Bounding Box Count (OptNMS): 117
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 122.31 ms
Cropped Image: 960 540

In [15]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 10.62 ms
Object Density: 0
Prediction time is: 10.62 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.04 ms
Object Density: 0
Prediction time is: 9.04 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.87 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  81
Final Bounding Box Count (OptNMS): 69
Adaptive Filtered Prediction:  20
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  6
Final Bounding Box Count (

In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp340/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.81s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.049
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [9]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 1024

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 697.89 ms
Object Density: 0
Prediction time is: 697.89 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.51 ms
Object Density: 0
Prediction time is: 9.51 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 43.17 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  1024
Slice Height:  1024
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 1
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  63
Final Bounding Box Count (OptNMS): 54
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 19.18 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.48 ms
Object Density: 12
********* Slice Parameters ***********
Slice Width:  1024
Slice Height: 

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp338/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.88s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.394
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.057
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp337/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.56s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.061
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [52]:
# Example usage: Adaptive-Optimized-NMS (NMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 12.43 ms
Object Density: 0
Prediction time is: 12.43 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.87 ms
Object Density: 0
Prediction time is: 9.87 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.18 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 112
POST PROCESS:  NMS
Original Prediction Count 110
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  3
Sliced Prediction time is: 726.82 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.50 ms
Object Density: 12
********* Slice Paramete

In [54]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp328/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.56s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.067
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.030
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [42]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 12.40 ms
Object Density: 0
Prediction time is: 12.40 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.17 ms
Object Density: 0
Prediction time is: 9.17 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.05 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  66
Final Bounding Box Count (OptNMS): 58
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  8
Final Bounding Box Coun

In [43]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp326/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.60s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.067
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [9]:
# Example usage: Adaptive-Optimized-NMS (NMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 1591.31 ms
Object Density: 0
Prediction time is: 1591.31 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 10.21 ms
Object Density: 0
Prediction time is: 10.21 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.04 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 464
POST PROCESS:  NMS
Original Prediction Count 105
Final Bounding Box Count (NMS):  43
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 2617.47 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.07 ms
Object Density: 12
********* Slice Parameters ***********
Slice Width:  64

In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp385/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.55s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
result_predict_nms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 1947
POST PROCESS:  NMS
Performing prediction on 1947 slices.


Performing inference on images:  10%|█         | 1/10 [00:11<01:43, 11.50s/it]

Original Prediction Count 177
Final Bounding Box Count (NMS):  40
Final Bounding Box Count (NMS):  53
Final Bounding Box Count (NMS):  11
Prediction time is: 11297.43 ms


...

Original Prediction Count 342
Final Bounding Box Count (NMS):  59
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  94
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  17
Prediction time is: 11305.31 ms


Performing inference on images:  90%|█████████ | 9/10 [01:43<00:11, 11.53s/it]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 1947
POST PROCESS:  NMS
Performing prediction on 1947 slices.


Performing inference on images: 100%|██████████| 10/10 [01:55<00:00, 11.53s/it]

Original Prediction Count 67
Final Bounding Box Count (NMS):  27
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  7
Prediction time is: 11238.10 ms
Prediction results are successfully exported to runs/predict/exp273
Model loaded in 0.31165361404418945 seconds.
Slicing performed in 0.2871568202972412 seconds.
Prediction performed in 113.2109706401825 seconds.
Exporting performed in 0.0768270492553711 seconds.


In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp273/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.42s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.059
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 10.87 ms
Object Density: 0
Prediction time is: 10.87 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.44 ms
Object Density: 0
Prediction time is: 9.44 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.16 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 464
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  63
Final Bounding Box Count (OptNMS): 55
Adaptive Filtered Prediction:  35
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  1
Final Bounding Box Count (

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp386/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.58s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.061
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [36]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 12.18 ms
Object Density: 0
Prediction time is: 12.18 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 10.01 ms
Object Density: 0
Prediction time is: 10.01 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.27 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  242
Final Bounding Box Count (OptNMS): 117
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 122.55 ms
Cropped Image: 960 

In [37]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp325/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.89s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.394
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.057
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [13]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 10.61 ms
Object Density: 0
Prediction time is: 10.61 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.07 ms
Object Density: 0
Prediction time is: 9.07 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.92 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 464
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  63
Final Bounding Box Count (OptNMS): 55
Adaptive Filtered Prediction:  35
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  1
Final Bounding Box Count (

In [14]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp339/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.58s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.061
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
#min_iou for small objects
# Example usage: Adaptive-Optimized-NMS (NMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 10.84 ms
Object Density: 0
Prediction time is: 10.84 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.27 ms
Object Density: 0
Prediction time is: 9.27 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.03 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  81
Final Bounding Box Count (OptNMS): 58
Adaptive Filtered Prediction:  20
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  6
Final Bounding Box Count 

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp336/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.80s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.072
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.049
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [29]:
# Example usage: Adaptive-Optimized-NMS (NMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 9.96 ms
Object Density: 0
Prediction time is: 9.96 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.05 ms
Object Density: 0
Prediction time is: 9.05 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.90 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 111
Final Bounding Box Count (NMS):  50
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Sliced Prediction time is: 252.38 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.40 ms
Object Density: 12
********* Slice Parameters ***********
Slice Width:  256
Slice H

In [30]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp324/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.76s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.033
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [24]:
# Example usage: Adaptive-Optimized-NMS (NMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 70.40 ms
Object Density: 0
Prediction time is: 70.40 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.01 ms
Object Density: 0
Prediction time is: 9.01 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.82 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 111
Final Bounding Box Count (NMS):  50
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Sliced Prediction time is: 248.38 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.31 ms
Object Density: 12
********* Slice Parameters ***********
Slice Width:  256
Slice

In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp322/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.76s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.033
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [10]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 10 image(s)...
*****************************************
File Name 0000074_03738_d_0000007
Cropped Image: 0 0 960 540
Initial Prediction time is: 474.66 ms
Object Density: 0
Prediction time is: 474.66 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 10.43 ms
Object Density: 0
Prediction time is: 10.43 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 53.68 ms
Object Density: 64
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  81
Final Bounding Box Count (OptNMS): 69
Adaptive Filtered Prediction:  20
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  6
Final Bounding Box Co

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp321/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.81s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.049
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [47]:
result_predict_optnms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Performing prediction on 112 slices.


Performing inference on images:  10%|█         | 1/10 [00:01<00:10,  1.21s/it]

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  22
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  49
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Prediction time is: 1013.86 ms


...

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  42
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Prediction time is: 964.06 ms
Image Name: 0000074_01218_d_0000002
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Performing prediction on 112 slices.


Performing inference on images:  90%|█████████ | 9/10 [00:10<00:01,  1.15s/it]

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  61
Final Bounding Box Count (OptNMS): 40
Adaptive Filtered Prediction:  44
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 11
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Prediction time is: 957.91 ms
Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Performing prediction on 112 slices.


Performing inference on images: 100%|██████████| 10/10 [00:11<00:00,  1.16s/it]

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  74
Final Bounding Box Count (OptNMS): 24
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 8
Prediction time is: 952.72 ms
Prediction results are successfully exported to runs/predict/exp242
Model loaded in 0.03381228446960449 seconds.
Slicing performed in 0.02675628662109375 seconds.
Prediction performed in 9.678879261016846 seconds.
Exporting performed in 0.07856059074401855 seconds.


In [48]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp242/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [17]:
result_predict_nms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 1024,
                         slice_width = 1024,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 6
POST PROCESS:  NMS
Performing prediction on 6 slices.


Performing inference on images:  10%|█         | 1/10 [00:00<00:05,  1.79it/s]

Original Prediction Count 408
Final Bounding Box Count (NMS):  72
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Prediction time is: 356.47 ms


...

Original Prediction Count 495
Final Bounding Box Count (NMS):  94
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Prediction time is: 305.33 ms


Performing inference on images:  90%|█████████ | 9/10 [00:04<00:00,  1.99it/s]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 6
POST PROCESS:  NMS
Performing prediction on 6 slices.


Performing inference on images: 100%|██████████| 10/10 [00:05<00:00,  1.98it/s]

Original Prediction Count 86
Final Bounding Box Count (NMS):  32
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Prediction time is: 286.29 ms
Prediction results are successfully exported to runs/predict/exp248
Model loaded in 0.25328826904296875 seconds.
Slicing performed in 0.02249765396118164 seconds.
Prediction performed in 3.0713741779327393 seconds.
Exporting performed in 0.07621550559997559 seconds.


In [18]:
#USING (NMS) METHOD
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp248/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.70s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.238
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.176
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.959
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
result_predict_nms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images:   0%|          | 0/10 [00:01<?, ?it/s]

Original Prediction Count 284
Final Bounding Box Count (NMS):  45
Final Bounding Box Count (NMS):  61
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Prediction time is: 1034.86 ms


...

Original Prediction Count 521
Final Bounding Box Count (NMS):  82
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  73
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  8
Prediction time is: 983.52 ms


Performing inference on images:  90%|█████████ | 9/10 [00:10<00:01,  1.20s/it]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images: 100%|██████████| 10/10 [00:11<00:00,  1.20s/it]

Original Prediction Count 361
Final Bounding Box Count (NMS):  27
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  78
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  14
Prediction time is: 974.52 ms
Prediction results are successfully exported to runs/predict/exp237
Model loaded in 0.03204154968261719 seconds.
Slicing performed in 0.02713918685913086 seconds.
Prediction performed in 9.924681186676025 seconds.
Exporting performed in 0.07428264617919922 seconds.


In [15]:
#USING (NMS) METHOD
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp237/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.89s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.061
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [33]:
result_predict_nms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Original Prediction Count 548
Final Bounding Box Count (NMS):  76
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Prediction time is: 530.85 ms


...

Original Prediction Count 638
Final Bounding Box Count (NMS):  106
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  4
Prediction time is: 478.58 ms


Performing inference on images:  90%|█████████ | 9/10 [00:06<00:00,  1.45it/s]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.


Performing inference on images: 100%|██████████| 10/10 [00:06<00:00,  1.45it/s]

Original Prediction Count 174
Final Bounding Box Count (NMS):  27
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  25
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  3
Prediction time is: 457.27 ms
Prediction results are successfully exported to runs/predict/exp239
Model loaded in 0.03331446647644043 seconds.
Slicing performed in 0.02303314208984375 seconds.
Prediction performed in 4.844449281692505 seconds.
Exporting performed in 0.07358455657958984 seconds.


In [34]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp239/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.89s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.149
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.224
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [38]:
result_predict_nms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images:   0%|          | 0/10 [00:03<?, ?it/s]

Original Prediction Count 379
Final Bounding Box Count (NMS):  47
Final Bounding Box Count (NMS):  100
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  3
Prediction time is: 3076.52 ms


...

Original Prediction Count 621
Final Bounding Box Count (NMS):  72
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  145
Final Bounding Box Count (NMS):  28
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  46
Final Bounding Box Count (NMS):  8
Prediction time is: 3168.26 ms


Performing inference on images:  90%|█████████ | 9/10 [00:29<00:03,  3.31s/it]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images: 100%|██████████| 10/10 [00:32<00:00,  3.27s/it]

Original Prediction Count 225
Final Bounding Box Count (NMS):  28
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  48
Final Bounding Box Count (NMS):  19
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  12
Prediction time is: 2957.10 ms
Prediction results are successfully exported to runs/predict/exp240
Model loaded in 0.035997867584228516 seconds.
Slicing performed in 0.17872071266174316 seconds.
Prediction performed in 30.577821731567383 seconds.
Exporting performed in 0.07458186149597168 seconds.


In [40]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp240/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.62s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.056
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [49]:
result_predict_nms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 464
POST PROCESS:  OptNMS
Performing prediction on 464 slices.


Performing inference on images:   0%|          | 0/10 [00:03<?, ?it/s]

Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  64
Final Bounding Box Count (OptNMS): 49
Adaptive Filtered Prediction:  183
Final Bounding Box Count (OptNMS): 81
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Prediction time is: 3044.42 ms


...

Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  79
Final Bounding Box Count (OptNMS): 64
Adaptive Filtered Prediction:  230
Final Bounding Box Count (OptNMS): 118
Adaptive Filtered Prediction:  60
Final Bounding Box Count (OptNMS): 26
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  81
Final Bounding Box Count (OptNMS): 40
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 4
Prediction time is: 3012.84 ms


Performing inference on images:  90%|█████████ | 9/10 [00:29<00:03,  3.27s/it]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 464
POST PROCESS:  OptNMS
Performing prediction on 464 slices.


Performing inference on images: 100%|██████████| 10/10 [00:32<00:00,  3.25s/it]

Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  26
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  101
Final Bounding Box Count (OptNMS): 46
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 14
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 12
Prediction time is: 2949.23 ms
Prediction results are successfully exported to runs/predict/exp244
Model loaded in 0.03511953353881836 seconds.
Slicing performed in 0.03945446014404297 seconds.
Prediction performed in 30.360904455184937 seconds.
Exporting performed in 0.07961559295654297 seconds.


In [50]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp244/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.24s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Prediction & Evaluation (15 Images)**

In [24]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 8.58 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  172
Final Bounding Box Count (OptNMS): 87
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 73.28 ms
Cropped Image: 480 0 960 270
Initial Prediction time is: 6.61 ms
Object Density: 4
Prediction time is: 6.61 ms
Cropped Image: 0 270 480 540
Initial Prediction time is: 6.97 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Rati

In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp334/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.59s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.240
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.230
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 7.93 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  4
IoU threshold (OptNMS): tensor([0.8000, 0.8000, 0.8000])
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  172
IoU threshold (OptNMS): tensor([0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000,
        0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.800

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp330/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.59s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.240
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.230
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [9]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 424.28 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  NMS
Original Prediction Count 187
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  50
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 70.98 ms
Cropped Image: 480 0 960 270
Initial Prediction time is: 6.51 ms
Object Density: 4
Prediction time is: 6.51 ms
Cropped Image: 0 270 480 540
Initial Prediction time is: 6.79 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  NMS
Original Prediction Count 90
Final Bounding Box Count (NMS):  7
Final Boundin

In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp331/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.54s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
result_predict_nms = predict(source='./test_vis_data',
                         dataset_json_path = './subset_vis_test_data_15.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000207_00300_d_0000004
Image Size:  (960, 540)
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.


Performing inference on images:   7%|▋         | 1/15 [00:00<00:05,  2.46it/s]

Original Prediction Count 454
Final Bounding Box Count (NMS):  17
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  81
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  6
Prediction time is: 333.29 ms
Image Name: 0000074_07850_d_0000015
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


...

Original Prediction Count 232
Final Bounding Box Count (NMS):  67
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  17
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  10
Prediction time is: 263.28 ms
Image Name: 0000192_00522_d_0000213
Image Size:  (960, 540)
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:07<00:00,  2.56it/s]

Original Prediction Count 221
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  28
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  14
Prediction time is: 259.66 ms
Image Name: 0000078_06777_d_0000020
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 15/15 [00:07<00:00,  1.96it/s]

Original Prediction Count 46
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  1
Prediction time is: 432.32 ms
Prediction results are successfully exported to runs/predict/exp246
Model loaded in 0.042932987213134766 seconds.
Slicing performed in 0.016870975494384766 seconds.
Prediction performed in 6.211194038391113 seconds.
Exporting performed in 0.04362964630126953 seconds.


In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './runs/predict/exp246/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.59s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.275
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.264
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.296
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
result_predict_nms_64 = predict(source='./test_vis_data',
                         dataset_json_path = './subset_vis_test_data_15.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000207_00300_d_0000004
Image Size:  (960, 540)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images:   7%|▋         | 1/15 [00:03<00:41,  2.99s/it]

Original Prediction Count 570
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  201
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  20
Prediction time is: 2894.03 ms
Image Name: 0000074_07850_d_0000015
Image Size:  (1920, 1080)
Sliced Boxes Count: 1947
POST PROCESS:  NMS
Performing prediction on 1947 slices.


...

Original Prediction Count 191
Final Bounding Box Count (NMS):  63
Final Bounding Box Count (NMS):  54
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  4
Prediction time is: 2645.06 ms
Image Name: 0000192_00522_d_0000213
Image Size:  (960, 540)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images:  93%|█████████▎| 14/15 [01:04<00:03,  3.54s/it]

Original Prediction Count 382
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  145
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  9
Prediction time is: 2658.15 ms
Image Name: 0000078_06777_d_0000020
Image Size:  (1360, 765)
Sliced Boxes Count: 966
POST PROCESS:  NMS
Performing prediction on 966 slices.


Performing inference on images: 100%|██████████| 15/15 [01:10<00:00,  4.68s/it]

Original Prediction Count 28
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  4
Prediction time is: 5458.92 ms
Prediction results are successfully exported to runs/predict/exp274
Model loaded in 0.2211778163909912 seconds.
Slicing performed in 0.1579134464263916 seconds.
Prediction performed in 68.62645077705383 seconds.
Exporting performed in 0.04335784912109375 seconds.


In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './runs/predict/exp274/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.53s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.176
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.440
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.044
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 9.30 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 45
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  54
Final Bounding Box Count (OptNMS): 51
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 270.55 ms
Cropped Image: 480 0 960 270
Initial Prediction time is: 6.40 ms
Object Density: 4
Prediction time is: 6.40 ms
Cropped Image: 0 270 480 540
Initial Prediction time is: 6.44 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64

In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp387/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.234
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.348
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [21]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)



Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 9.12 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 45
POST PROCESS:  NMS
Original Prediction Count 60
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 268.66 ms
Cropped Image: 480 0 960 270
Initial Prediction time is: 6.67 ms
Object Density: 4
Prediction time is: 6.67 ms
Cropped Image: 0 270 480 540
Initial Prediction time is: 6.39 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 45
POST PROCESS:  NMS
Original Prediction Count 42
Final Bounding Box

In [22]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp388/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.48s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.232
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
result_predict_nms_128 = predict(source='./test_vis_data',
                         dataset_json_path = './subset_vis_test_data_15.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000207_00300_d_0000004
Image Size:  (960, 540)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images:   7%|▋         | 1/15 [00:00<00:12,  1.08it/s]

Original Prediction Count 413
Final Bounding Box Count (NMS):  77
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  68
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  4
Prediction time is: 843.84 ms
Image Name: 0000074_07850_d_0000015
Image Size:  (1920, 1080)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


...

Original Prediction Count 259
Final Bounding Box Count (NMS):  64
Final Bounding Box Count (NMS):  76
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Prediction time is: 760.74 ms
Image Name: 0000192_00522_d_0000213
Image Size:  (960, 540)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:19<00:01,  1.05s/it]

Original Prediction Count 193
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  51
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  10
Prediction time is: 756.59 ms
Image Name: 0000078_06777_d_0000020
Image Size:  (1360, 765)
Sliced Boxes Count: 231
POST PROCESS:  NMS
Performing prediction on 231 slices.


Performing inference on images: 100%|██████████| 15/15 [00:20<00:00,  1.39s/it]

Original Prediction Count 299
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  96
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  1
Prediction time is: 1559.45 ms
Prediction results are successfully exported to runs/predict/exp247
Model loaded in 0.2223191261291504 seconds.
Slicing performed in 0.0251924991607666 seconds.
Prediction performed in 19.304105520248413 seconds.
Exporting performed in 0.04598522186279297 seconds.


In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './runs/predict/exp247/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.49s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.183
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.102
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.329
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.230
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 10.50 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 15
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  17
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  105
Final Bounding Box Count (OptNMS): 66
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 120.15 ms
Cropped Image: 480 0 960 270
Initial Prediction time is: 7.07 ms
Object Density: 4
Prediction time is: 7.07 ms
Cropped Image: 0 270 480 540
Initial Prediction time is: 6.72 

In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp332/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.49s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.122
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.215
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.122
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.172
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.348
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [21]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 9.59 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 15
POST PROCESS:  NMS
Original Prediction Count 132
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  45
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 118.64 ms
Cropped Image: 480 0 960 270
Initial Prediction time is: 7.02 ms
Object Density: 4
Prediction time is: 7.02 ms
Cropped Image: 0 270 480 540
Initial Prediction time is: 7.12 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 15
POST PROCESS:  NMS
Origi

In [22]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp333/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.47s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.217
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
#iou_min for small objects
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000207_00300_d_0000004
Cropped Image: 0 0 480 270
Initial Prediction time is: 472.24 ms
Object Density: 17
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  172
Final Bounding Box Count (OptNMS): 54
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 72.55 ms
Cropped Image: 480 0 960 270
Initial Prediction time is: 6.61 ms
Object Density: 4
Prediction time is: 6.61 ms
Cropped Image: 0 270 480 540
Initial Prediction time is: 6.90 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ra

In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp335/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.56s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [1]:
!python generate_subset.py

Copied 9999938_00000_d_0000028.jpg to test_subsets/v1/images
Copied 0000078_06506_d_0000018.jpg to test_subsets/v1/images
Copied 9999979_00000_d_0000034.jpg to test_subsets/v1/images
Copied 0000310_03500_d_0000125.jpg to test_subsets/v1/images
Copied 9999963_00000_d_0000040.jpg to test_subsets/v1/images
Copied 9999952_00000_d_0000001.jpg to test_subsets/v1/images
Copied 9999952_00000_d_0000262.jpg to test_subsets/v1/images
Copied 9999938_00000_d_0000288.jpg to test_subsets/v1/images
Copied 9999938_00000_d_0000035.jpg to test_subsets/v1/images
Copied 9999973_00000_d_0000016.jpg to test_subsets/v1/images
Copied 9999938_00000_d_0000364.jpg to test_subsets/v1/images
Copied 0000105_02201_d_0000071.jpg to test_subsets/v1/images
Copied 9999938_00000_d_0000030.jpg to test_subsets/v1/images
Copied 9999973_00000_d_0000149.jpg to test_subsets/v1/images
Copied 9999973_00000_d_0000031.jpg to test_subsets/v1/images
Subset creation completed.


#### **Subset of 15 images**

In [10]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


 Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 483.45 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 12
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 7
Sliced Prediction time is: 79.69 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.88 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive 

In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp341/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.199
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.308
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [10]:
#low area - low IoU
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 475.28 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 11
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Sliced Prediction time is: 79.00 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.80 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive F

In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp355/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.199
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [13]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.07 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 30
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 79.08 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 8.72 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 65
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  3
Final Bounding Box 

In [14]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp342/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.13s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.199
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.306
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.191
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
result_predict_nms = predict(source='./test_subsets/v1',
                         dataset_json_path = './subset_visdrone_test_data_15_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999952_00000_d_0000001
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images:   7%|▋         | 1/15 [00:00<00:10,  1.37it/s]

Original Prediction Count 231
Final Bounding Box Count (NMS):  31
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  10
Prediction time is: 607.24 ms
Image Name: 9999938_00000_d_0000364
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


...

Original Prediction Count 185
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  43
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  5
Prediction time is: 533.91 ms
Image Name: 0000310_03500_d_0000125
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:10<00:00,  1.55it/s]

Original Prediction Count 77
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Prediction time is: 443.27 ms
Image Name: 9999938_00000_d_0000028
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images: 100%|██████████| 15/15 [00:11<00:00,  1.29it/s]

Original Prediction Count 253
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  39
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Prediction time is: 545.98 ms
Prediction results are successfully exported to runs/predict/exp250
Model loaded in 0.03951668739318848 seconds.
Slicing performed in 0.025684833526611328 seconds.
Prediction performed in 9.548431158065796 seconds.
Exporting performed in 0.053870439529418945 seconds.


In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './runs/predict/exp250/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.158
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.271
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.185
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.295
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.572
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.096
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
result_predict_nms = predict(source='./test_subsets/v1',
                         dataset_json_path = './subset_visdrone_test_data_15_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999952_00000_d_0000001
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images:   7%|▋         | 1/15 [00:01<00:26,  1.91s/it]

Original Prediction Count 341
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  106
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  4
Prediction time is: 1782.13 ms
Image Name: 9999938_00000_d_0000364
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


...

Original Prediction Count 353
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  88
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  8
Prediction time is: 1710.61 ms
Image Name: 0000310_03500_d_0000125
Image Size:  (1360, 765)
Sliced Boxes Count: 231
POST PROCESS:  NMS
Performing prediction on 231 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:30<00:01,  1.88s/it]

Original Prediction Count 122
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  26
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  3
Prediction time is: 1534.66 ms
Image Name: 9999938_00000_d_0000028
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images: 100%|██████████| 15/15 [00:32<00:00,  2.18s/it]

Original Prediction Count 444
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  112
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  31
Final Bounding Box Count (NMS):  3
Prediction time is: 1724.30 ms
Prediction results are successfully exported to runs/predict/exp251
Model loaded in 0.2147235870361328 seconds.
Slicing performed in 0.03839731216430664 seconds.
Prediction performed in 30.522034883499146 seconds.
Exporting performed in 0.05655980110168457 seconds.


In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './runs/predict/exp251/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.263
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.558
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [20]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.82 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 53
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 202.25 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.87 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 79
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  33
Final Bounding B

In [21]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp343/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.180
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.52 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  38
Final Bounding Box Count (OptNMS): 29
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 195.59 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.57 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ra

In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp356/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.091
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.166
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.291
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.047
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.53 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  51
Final Bounding Box Count (OptNMS): 46
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 10
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Sliced Prediction time is: 547.29 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.60 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Rati

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp357/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.191
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.114
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.203
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [23]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 11.04 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  38
Final Bounding Box Count (OptNMS): 30
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 204.77 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 8.03 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ra

In [24]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp344/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.168
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.106
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.291
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [26]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.81 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  38
Final Bounding Box Count (OptNMS): 30
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 204.19 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.85 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ra

In [27]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp345/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.168
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.106
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.291
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [10]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 484.60 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  51
Final Bounding Box Count (OptNMS): 48
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 11
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Sliced Prediction time is: 570.88 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.82 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Rat

In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp347/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.191
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.113
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.326
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [15]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             
    dataset_json_path=json_path,           
    detection_model=detection_model,
    base_slice_size=slice_size             
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.35 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 72
Final Bounding Box Count (NMS):  46
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  3
Sliced Prediction time is: 570.42 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.92 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 65
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  43
Final Bounding Box C

In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp348/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.113
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
result_predict_nms = predict(source='./test_subsets/v1',
                         dataset_json_path = './subset_visdrone_test_data_15_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999952_00000_d_0000001
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images:   7%|▋         | 1/15 [00:06<01:32,  6.58s/it]

Original Prediction Count 447
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  143
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  8
Prediction time is: 6442.04 ms
Image Name: 9999938_00000_d_0000364
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


...

Original Prediction Count 93
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Prediction time is: 5865.80 ms
Image Name: 9999938_00000_d_0000028
Image Size:  (1400, 788)
Sliced Boxes Count: 1032


Performing inference on images:  93%|█████████▎| 14/15 [01:50<00:06,  6.79s/it]

POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images: 100%|██████████| 15/15 [01:57<00:00,  7.82s/it]

Original Prediction Count 354
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  130
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  27
Final Bounding Box Count (NMS):  6
Prediction time is: 6463.45 ms
Prediction results are successfully exported to runs/predict/exp254
Model loaded in 0.046219587326049805 seconds.
Slicing performed in 0.18907880783081055 seconds.
Prediction performed in 115.21478796005249 seconds.
Exporting performed in 0.057504892349243164 seconds.


In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './runs/predict/exp254/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.037
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.302
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.572
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [28]:
result_predict_nms = predict(source='./test_subsets/v1',
                         dataset_json_path = './subset_visdrone_test_data_15_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999952_00000_d_0000001
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images:   7%|▋         | 1/15 [00:01<00:26,  1.89s/it]

Original Prediction Count 341
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  106
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  4
Prediction time is: 1762.99 ms
Image Name: 9999938_00000_d_0000364
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


...

Original Prediction Count 353
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  88
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  8
Prediction time is: 1700.86 ms
Image Name: 0000310_03500_d_0000125
Image Size:  (1360, 765)
Sliced Boxes Count: 231
POST PROCESS:  NMS
Performing prediction on 231 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:30<00:01,  1.87s/it]

Original Prediction Count 122
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  26
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  3
Prediction time is: 1523.21 ms
Image Name: 9999938_00000_d_0000028
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images: 100%|██████████| 15/15 [00:32<00:00,  2.18s/it]

Original Prediction Count 444
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  112
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  31
Final Bounding Box Count (NMS):  3
Prediction time is: 1712.83 ms
Prediction results are successfully exported to runs/predict/exp252
Model loaded in 0.31133413314819336 seconds.
Slicing performed in 0.039160966873168945 seconds.
Prediction performed in 30.452358961105347 seconds.
Exporting performed in 0.05643749237060547 seconds.


In [29]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './runs/predict/exp252/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.263
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.558
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [30]:
result_predict_nms = predict(source='./test_subsets/v1',
                         dataset_json_path = './subset_visdrone_test_data_15_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999952_00000_d_0000001
Image Size:  (1400, 788)
Sliced Boxes Count: 15
POST PROCESS:  NMS
Performing prediction on 15 slices.


Performing inference on images:   7%|▋         | 1/15 [00:00<00:05,  2.34it/s]

Original Prediction Count 144
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  27
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  6
Prediction time is: 312.01 ms
Image Name: 9999938_00000_d_0000364
Image Size:  (1400, 788)
Sliced Boxes Count: 15
POST PROCESS:  NMS
Performing prediction on 15 slices.


...

Original Prediction Count 136
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Prediction time is: 251.06 ms
Image Name: 0000310_03500_d_0000125
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  NMS
Performing prediction on 10 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:06<00:00,  2.94it/s]

Original Prediction Count 63
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Prediction time is: 173.03 ms
Image Name: 9999938_00000_d_0000028
Image Size:  (1400, 788)
Sliced Boxes Count: 15
POST PROCESS:  NMS
Performing prediction on 15 slices.


Performing inference on images: 100%|██████████| 15/15 [00:06<00:00,  2.35it/s]

Original Prediction Count 144
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  4
Prediction time is: 252.65 ms
Prediction results are successfully exported to runs/predict/exp253
Model loaded in 0.1742861270904541 seconds.
Slicing performed in 0.021878480911254883 seconds.
Prediction performed in 4.391990423202515 seconds.
Exporting performed in 0.05170178413391113 seconds.


In [31]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './runs/predict/exp253/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.183
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.312
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.209
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.443
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.527
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [32]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v1/images'
json_path = "./subset_visdrone_test_data_15_v1.json"
slice_size = 512

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999952_00000_d_0000001
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.91 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  16
Final Bounding Box Count (OptNMS): 10
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 212.67 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.90 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive 

In [33]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp346/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.12s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.221
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.162
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.337
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.494
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 15 images (V2)**

In [24]:
result_predict_nms = predict(source='./test_subsets/v3',
                         dataset_json_path = './subset_visdrone_test_data_15_v2.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999973_00000_d_0000008
Image Size:  (1916, 1078)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Original Prediction Count 107
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  23
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Prediction time is: 971.39 ms


...

Original Prediction Count 117
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  17
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  2
Prediction time is: 532.47 ms
Image Name: 9999952_00000_d_0000235
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:10<00:00,  1.42it/s]

Original Prediction Count 127
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  26
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Prediction time is: 535.23 ms
Image Name: 9999938_00000_d_0000384
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images: 100%|██████████| 15/15 [00:11<00:00,  1.32it/s]

Original Prediction Count 557
Final Bounding Box Count (NMS):  95
Final Bounding Box Count (NMS):  55
Final Bounding Box Count (NMS):  17
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  5
Prediction time is: 583.10 ms
Prediction results are successfully exported to runs/predict/exp260
Model loaded in 0.029228925704956055 seconds.
Slicing performed in 0.02398514747619629 seconds.
Prediction performed in 9.34496259689331 seconds.
Exporting performed in 0.057511329650878906 seconds.


In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './runs/predict/exp260/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.176
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.112
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.301
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.698
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [26]:
# Example usage: Adaptive-Optimized-NMS Take 3(Adaptive-OptNMS)
source_folder = './test_subsets/v3/images'
json_path = "./subset_visdrone_test_data_15_v2.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999973_00000_d_0000008
Cropped Image: 0 0 958 539
Initial Prediction time is: 71.34 ms
Object Density: 14
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 15
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  30
Final Bounding Box Count (OptNMS): 13
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 139.89 ms
Cropped Image: 958 0 1916 539
Initial Prediction time is: 9.71 ms
Object Density: 4
Prediction time is: 9.71 ms
Cropped Image: 0 539 958 1078
Initial Prediction time is: 9.29 ms

In [27]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './sliced_predictions/exp358/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.231
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.279
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [30]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v3/images'
json_path = "./subset_visdrone_test_data_15_v2.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999973_00000_d_0000008
Cropped Image: 0 0 958 539
Initial Prediction time is: 10.57 ms
Object Density: 14
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 15
POST PROCESS:  NMS
Original Prediction Count 42
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 137.97 ms
Cropped Image: 958 0 1916 539
Initial Prediction time is: 9.29 ms
Object Density: 4
Prediction time is: 9.29 ms
Cropped Image: 0 539 958 1078
Initial Prediction time is: 8.78 ms
Object Density: 0
Prediction time is: 8.78 ms
Cropped Image: 958 539 1916 1078
Initial Prediction time is: 9.12 ms
Object Density: 3
Prediction time is: 9.12 ms
____________________________________

In [31]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './sliced_predictions/exp360/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.232
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.281
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.112
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [9]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v3/images'
json_path = "./subset_visdrone_test_data_15_v2.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999973_00000_d_0000008
Cropped Image: 0 0 958 539
Initial Prediction time is: 511.93 ms
Object Density: 14
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 45
POST PROCESS:  NMS
Original Prediction Count 29
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 297.53 ms
Cropped Image: 958 0 1916 539
Initial Prediction time is: 9.56 ms
Object Density: 4
Prediction time is: 9.56 ms
Cropped Image: 0 539 958 1078
Initial Prediction time is: 8.95 ms
Object Density: 0
Prediction time is: 8.95 ms
Cropped Image: 958 539 1916 1078
Initial Prediction time is: 8.93 ms
Object Density: 3
Prediction time is: 8.93 ms


In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './sliced_predictions/exp361/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.230
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.274
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.106
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v3/images'
json_path = "./subset_visdrone_test_data_15_v2.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999973_00000_d_0000008
Cropped Image: 0 0 958 539
Initial Prediction time is: 12.35 ms
Object Density: 14
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 45
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  15
Final Bounding Box Count (OptNMS): 15
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 298.90 ms
Cropped Image: 958 0 1916 539
Initial Prediction time is: 9.34 ms
Object Density: 4
Prediction time is: 9.3

In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './sliced_predictions/exp362/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.229
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.273
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.106
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
result_predict_nms = predict(source='./test_subsets/v3',
                         dataset_json_path = './subset_visdrone_test_data_15_v2.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999973_00000_d_0000008
Image Size:  (1916, 1078)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images:   0%|          | 0/15 [00:03<?, ?it/s]

Original Prediction Count 163
Final Bounding Box Count (NMS):  19
Final Bounding Box Count (NMS):  51
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  11
Prediction time is: 3037.14 ms


...

Original Prediction Count 163
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  48
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  3
Prediction time is: 1615.87 ms
Image Name: 9999952_00000_d_0000235
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:29<00:01,  1.90s/it]

Original Prediction Count 264
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  66
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  3
Prediction time is: 1629.42 ms
Image Name: 9999938_00000_d_0000384
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images: 100%|██████████| 15/15 [00:31<00:00,  2.07s/it]

Original Prediction Count 773
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  176
Final Bounding Box Count (NMS):  57
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  40
Prediction time is: 1689.13 ms
Prediction results are successfully exported to runs/predict/exp261
Model loaded in 0.06559562683105469 seconds.
Slicing performed in 0.037787437438964844 seconds.
Prediction performed in 28.902344942092896 seconds.
Exporting performed in 0.06074810028076172 seconds.


In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './runs/predict/exp261/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.38s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.066
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.106
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.294
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.698
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [10]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v3/images'
json_path = "./subset_visdrone_test_data_15_v2.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)



Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999973_00000_d_0000008
Cropped Image: 0 0 958 539
Initial Prediction time is: 10.52 ms
Object Density: 14
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 180
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 11
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 1040.82 ms
Cropped Image: 958 0 1916 539
Initial Prediction time is: 9.18 ms
Object Density: 4
Prediction time is: 9.1

In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './sliced_predictions/exp364/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.243
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.278
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.261
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
result_predict_nms = predict(source='./test_subsets/v3',
                         dataset_json_path = './subset_visdrone_test_data_15_v2.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999973_00000_d_0000008
Image Size:  (1916, 1078)
Sliced Boxes Count: 1947
POST PROCESS:  NMS
Performing prediction on 1947 slices.


Performing inference on images:   0%|          | 0/15 [00:11<?, ?it/s]

Original Prediction Count 145
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  71
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  7
Prediction time is: 11649.36 ms


...

Original Prediction Count 170
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  58
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  11
Prediction time is: 6006.62 ms
Image Name: 9999938_00000_d_0000384
Image Size:  (1400, 788)
Sliced Boxes Count: 1032


Performing inference on images:  93%|█████████▎| 14/15 [01:44<00:06,  6.73s/it]

POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images: 100%|██████████| 15/15 [01:50<00:00,  7.36s/it]

Original Prediction Count 772
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  226
Final Bounding Box Count (NMS):  56
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  25
Prediction time is: 6292.13 ms
Prediction results are successfully exported to runs/predict/exp262
Model loaded in 0.06565284729003906 seconds.
Slicing performed in 0.1810767650604248 seconds.
Prediction performed in 108.3421220779419 seconds.
Exporting performed in 0.06313514709472656 seconds.


In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './runs/predict/exp262/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.34s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.116
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.041
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.344
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.698
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v3/images'
json_path = "./subset_visdrone_test_data_15_v2.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999973_00000_d_0000008
Cropped Image: 0 0 958 539
Initial Prediction time is: 11.40 ms
Object Density: 14
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 180
POST PROCESS:  NMS
Original Prediction Count 23
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 1025.07 ms
Cropped Image: 958 0 1916 539
Initial Prediction time is: 9.07 ms
Object Density: 4
Prediction time is: 9.07 ms
Cropped Image: 0 539 958 1078
Initial Prediction time is: 8.62 ms
Object Density: 0
Prediction time is: 8.62 ms
Cropped Image: 958 539 1916 1078
Initial Prediction time is: 8.90 ms
Object Density: 3
Prediction time is: 8.90 ms
_

In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v2.json' --result_json_path './sliced_predictions/exp365/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.242
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.277
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.261
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 15 images (V4)**

In [19]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v4/images'
json_path = "./subset_visdrone_test_data_15_v3.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)



Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000073_00377_d_0000001
Cropped Image: 0 0 960 540
Initial Prediction time is: 9.63 ms
Object Density: 0
Prediction time is: 9.63 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 8.83 ms
Object Density: 0
Prediction time is: 8.83 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.58 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 161
Final Bounding Box Count (NMS):  60
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 246.99 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.13 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice H

In [20]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './sliced_predictions/exp367/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.111
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.371
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.765
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [21]:
result_predict_nms = predict(source='./test_subsets/v4',
                         dataset_json_path = './subset_visdrone_test_data_15_v3.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000073_00377_d_0000001
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images:   7%|▋         | 1/15 [00:01<00:17,  1.23s/it]

Original Prediction Count 412
Final Bounding Box Count (NMS):  76
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  48
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  4
Prediction time is: 1036.48 ms


...

Original Prediction Count 236
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  53
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Prediction time is: 528.52 ms
Image Name: 9999952_00000_d_0000278
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:10<00:00,  1.44it/s]

Original Prediction Count 32
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Prediction time is: 503.16 ms
Image Name: 9999938_00000_d_0000255
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images: 100%|██████████| 15/15 [00:11<00:00,  1.31it/s]

Original Prediction Count 149
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  11
Prediction time is: 519.50 ms
Prediction results are successfully exported to runs/predict/exp263
Model loaded in 0.30522608757019043 seconds.
Slicing performed in 0.02569413185119629 seconds.
Prediction performed in 9.454628467559814 seconds.
Exporting performed in 0.049965620040893555 seconds.


In [22]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './runs/predict/exp263/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.235
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.393
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.890
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [24]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v4/images'
json_path = "./subset_visdrone_test_data_15_v3.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000073_00377_d_0000001
Cropped Image: 0 0 960 540
Initial Prediction time is: 11.48 ms
Object Density: 0
Prediction time is: 11.48 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.29 ms
Object Density: 0
Prediction time is: 9.29 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.03 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  124
Final Bounding Box Count (OptNMS): 89
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 15
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  1
Final Bounding Box Count

In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './sliced_predictions/exp368/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.20s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.112
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.203
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.373
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.765
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.214
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [26]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v4/images'
json_path = "./subset_visdrone_test_data_15_v3.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000073_00377_d_0000001
Cropped Image: 0 0 960 540
Initial Prediction time is: 11.43 ms
Object Density: 0
Prediction time is: 11.43 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.22 ms
Object Density: 0
Prediction time is: 9.22 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.07 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  63
Final Bounding Box Count (OptNMS): 62
Adaptive Filtered Prediction:  69
Final Bounding Box Count (OptNMS): 38
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  8
Final Bounding Box Count

In [27]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './sliced_predictions/exp369/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.102
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.379
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.765
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [28]:
result_predict_nms = predict(source='./test_subsets/v4',
                         dataset_json_path = './subset_visdrone_test_data_15_v3.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000073_00377_d_0000001
Image Size:  (1920, 1080)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images:   0%|          | 0/15 [00:03<?, ?it/s]

Original Prediction Count 547
Final Bounding Box Count (NMS):  55
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  125
Final Bounding Box Count (NMS):  26
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  36
Final Bounding Box Count (NMS):  7
Prediction time is: 3092.77 ms


...

Original Prediction Count 363
Final Bounding Box Count (NMS):  24
Final Bounding Box Count (NMS):  84
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  2
Prediction time is: 1639.28 ms
Image Name: 9999952_00000_d_0000278
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:29<00:01,  1.94s/it]

Original Prediction Count 26
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  3
Prediction time is: 1570.91 ms
Image Name: 9999938_00000_d_0000255
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images: 100%|██████████| 15/15 [00:31<00:00,  2.11s/it]

Original Prediction Count 271
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  48
Final Bounding Box Count (NMS):  17
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  25
Final Bounding Box Count (NMS):  3
Prediction time is: 1622.35 ms
Prediction results are successfully exported to runs/predict/exp264
Model loaded in 0.14270281791687012 seconds.
Slicing performed in 0.0383601188659668 seconds.
Prediction performed in 29.615706205368042 seconds.
Exporting performed in 0.05160880088806152 seconds.


In [29]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './runs/predict/exp264/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.38s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.133
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.066
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.738
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.021
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [31]:

# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v4/images'
json_path = "./subset_visdrone_test_data_15_v3.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)



Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000073_00377_d_0000001
Cropped Image: 0 0 960 540
Initial Prediction time is: 10.41 ms
Object Density: 0
Prediction time is: 10.41 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.20 ms
Object Density: 0
Prediction time is: 9.20 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.92 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 112
POST PROCESS:  NMS
Original Prediction Count 161
Final Bounding Box Count (NMS):  48
Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  3
Sliced Prediction time is: 729.18 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.33 ms
Object Density: 15
********* Slice Parameters

In [32]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './sliced_predictions/exp370/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.102
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.184
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.100
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.377
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.765
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.047
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.216
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [33]:
result_predict_nms = predict(source='./test_subsets/v4',
                         dataset_json_path = './subset_visdrone_test_data_15_v3.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000073_00377_d_0000001
Image Size:  (1920, 1080)
Sliced Boxes Count: 1947
POST PROCESS:  NMS
Performing prediction on 1947 slices.


Performing inference on images:   0%|          | 0/15 [00:11<?, ?it/s]

Original Prediction Count 315
Final Bounding Box Count (NMS):  47
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  95
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  16
Prediction time is: 11700.66 ms


...

Original Prediction Count 221
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  66
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  1
Prediction time is: 6112.73 ms
Image Name: 9999952_00000_d_0000278
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images:  93%|█████████▎| 14/15 [01:48<00:07,  7.06s/it]

Original Prediction Count 16
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  3
Prediction time is: 6056.55 ms
Image Name: 9999938_00000_d_0000255
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images: 100%|██████████| 15/15 [01:54<00:00,  7.62s/it]

Original Prediction Count 184
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  46
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  1
Prediction time is: 6099.86 ms
Prediction results are successfully exported to runs/predict/exp265
Model loaded in 0.058290719985961914 seconds.
Slicing performed in 0.0855093002319336 seconds.
Prediction performed in 112.24269700050354 seconds.
Exporting performed in 0.05145883560180664 seconds.


In [34]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './runs/predict/exp265/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.44s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.077
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.988
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.021
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.248
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [35]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v4/images'
json_path = "./subset_visdrone_test_data_15_v3.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000073_00377_d_0000001
Cropped Image: 0 0 960 540
Initial Prediction time is: 11.69 ms
Object Density: 0
Prediction time is: 11.69 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.29 ms
Object Density: 0
Prediction time is: 9.29 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 10.06 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 464
POST PROCESS:  NMS
Original Prediction Count 136
Final Bounding Box Count (NMS):  47
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  3
Sliced Prediction time is: 2699.54 ms
Cropped Image: 960 540 1920 1080
Initial Prediction time is: 9.38 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  

In [36]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './sliced_predictions/exp371/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.25s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.193
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.394
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.765
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.050
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.227
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [38]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v4/images'
json_path = "./subset_visdrone_test_data_15_v3.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 0000073_00377_d_0000001
Cropped Image: 0 0 960 540
Initial Prediction time is: 11.47 ms
Object Density: 0
Prediction time is: 11.47 ms
Cropped Image: 960 0 1920 540
Initial Prediction time is: 9.19 ms
Object Density: 0
Prediction time is: 9.19 ms
Cropped Image: 0 540 960 1080
Initial Prediction time is: 9.97 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 464
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  61
Final Bounding Box Count (OptNMS): 61
Adaptive Filtered Prediction:  70
Final Bounding Box Count (OptNMS): 37
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Sliced Prediction time is: 2695.60 ms
Cropped Image: 960 540

In [39]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v3.json' --result_json_path './sliced_predictions/exp372/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.194
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.396
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.765
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.050
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.229
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 15 images (V5)**

In [53]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.65 ms
Object Density: 3
Prediction time is: 11.65 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.04 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 108.63 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.27 ms
Object Density: 7
Prediction time is: 8.27 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.26 ms
Object Density: 2
Prediction time is: 8.26 ms
___________________________________

In [54]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp374/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.238
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [48]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 67.21 ms
Object Density: 3
Prediction time is: 67.21 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.49 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  26
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 106.80 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.59 m

In [49]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp373/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.240
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [50]:
result_predict_nms = predict(source='./test_subsets/v5',
                         dataset_json_path = './subset_visdrone_test_data_15_v5.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999963_00000_d_0000057
Image Size:  (1400, 1050)
Sliced Boxes Count: 80
POST PROCESS:  NMS
Performing prediction on 80 slices.


Performing inference on images:   7%|▋         | 1/15 [00:00<00:12,  1.16it/s]

Original Prediction Count 120
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Prediction time is: 721.16 ms
Image Name: 9999963_00000_d_0000006
Image Size:  (1400, 1050)
Sliced Boxes Count: 80
POST PROCESS:  NMS
Performing prediction on 80 slices.


...

Original Prediction Count 221
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  45
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Prediction time is: 510.57 ms
Image Name: 9999952_00000_d_0000216
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:09<00:00,  1.54it/s]

Original Prediction Count 121
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  3
Prediction time is: 504.89 ms
Image Name: 9999938_00000_d_0000124
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images: 100%|██████████| 15/15 [00:10<00:00,  1.47it/s]

Original Prediction Count 46
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Prediction time is: 495.17 ms
Prediction results are successfully exported to runs/predict/exp266
Model loaded in 0.3077712059020996 seconds.
Slicing performed in 0.022861480712890625 seconds.
Prediction performed in 8.400869846343994 seconds.
Exporting performed in 0.05022716522216797 seconds.


In [51]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './runs/predict/exp266/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.254
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.185
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.650
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [55]:
result_predict_nms = predict(source='./test_subsets/v5',
                         dataset_json_path = './subset_visdrone_test_data_15_v5.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999963_00000_d_0000057
Image Size:  (1400, 1050)
Sliced Boxes Count: 336
POST PROCESS:  NMS
Performing prediction on 336 slices.


Performing inference on images:   7%|▋         | 1/15 [00:02<00:32,  2.33s/it]

Original Prediction Count 195
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  48
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  1
Prediction time is: 2183.90 ms
Image Name: 9999963_00000_d_0000006
Image Size:  (1400, 1050)
Sliced Boxes Count: 336
POST PROCESS:  NMS
Performing prediction on 336 slices.


...

Original Prediction Count 240
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  75
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Prediction time is: 1607.38 ms
Image Name: 9999952_00000_d_0000216
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:26<00:01,  1.82s/it]

Original Prediction Count 128
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  19
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  7
Prediction time is: 1592.54 ms
Image Name: 9999938_00000_d_0000124
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images: 100%|██████████| 15/15 [00:28<00:00,  1.91s/it]

Original Prediction Count 232
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  67
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  24
Final Bounding Box Count (NMS):  2
Prediction time is: 1623.33 ms
Prediction results are successfully exported to runs/predict/exp267
Model loaded in 0.3124072551727295 seconds.
Slicing performed in 0.03406977653503418 seconds.
Prediction performed in 26.766183614730835 seconds.
Exporting performed in 0.05249333381652832 seconds.


In [56]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './runs/predict/exp267/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.097
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.153
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.113
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.045
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.234
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.696
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [57]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 10.53 ms
Object Density: 3
Prediction time is: 10.53 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.70 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 35
POST PROCESS:  NMS
Original Prediction Count 38
Final Bounding Box Count (NMS):  25
Final Bounding Box Count (NMS):  6
Sliced Prediction time is: 227.60 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.17 ms
Object Density: 7
Prediction time is: 8.17 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.14 ms
Object Density: 2
Prediction time is: 8.14 ms
________________________________________________
*****************************************
File Name 9999963_00000_d_0000006
Cropped Image: 

In [58]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp375/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [60]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 10.19 ms
Object Density: 3
Prediction time is: 10.19 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.70 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 35
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 9
Sliced Prediction time is: 227.17 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.17 ms
Object Density: 7
Prediction time is: 8.17 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.17 ms
Object Density: 2
Prediction time is: 8.17 ms
_______________________________________________

In [61]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp376/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [62]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size             
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.23 ms
Object Density: 3
Prediction time is: 12.23 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.78 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 130
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 23
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 747.83 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.47 ms
Object Density: 7
Prediction time is: 8.47 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.40 ms
Object Density: 2
Prediction time is: 8.40 ms
________________________________________________

In [63]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp377/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.319
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [64]:
result_predict_nms = predict(source='./test_subsets/v5',
                         dataset_json_path = './subset_visdrone_test_data_15_v5.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999963_00000_d_0000057
Image Size:  (1400, 1050)
Sliced Boxes Count: 1376
POST PROCESS:  NMS
Performing prediction on 1376 slices.


Performing inference on images:   7%|▋         | 1/15 [00:08<01:55,  8.26s/it]

Original Prediction Count 105
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  45
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Prediction time is: 8118.80 ms
Image Name: 9999963_00000_d_0000006
Image Size:  (1400, 1050)
Sliced Boxes Count: 1376
POST PROCESS:  NMS
Performing prediction on 1376 slices.


...

Original Prediction Count 195
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  86
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  2
Prediction time is: 5950.86 ms
Image Name: 9999952_00000_d_0000216
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images:  93%|█████████▎| 14/15 [01:35<00:06,  6.52s/it]

Original Prediction Count 122
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  32
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  1
Prediction time is: 5989.63 ms
Image Name: 9999938_00000_d_0000124
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images: 100%|██████████| 15/15 [01:41<00:00,  6.79s/it]

Original Prediction Count 113
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  38
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  9
Prediction time is: 5990.96 ms
Prediction results are successfully exported to runs/predict/exp268
Model loaded in 0.0669240951538086 seconds.
Slicing performed in 0.16933774948120117 seconds.
Prediction performed in 99.93511056900024 seconds.
Exporting performed in 0.05110359191894531 seconds.


In [65]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './runs/predict/exp268/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.25s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.175
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.267
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.696
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [67]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.50 ms
Object Density: 3
Prediction time is: 12.50 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.94 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 130
POST PROCESS:  NMS
Original Prediction Count 25
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 751.76 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.35 ms
Object Density: 7
Prediction time is: 8.35 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.35 ms
Object Density: 2
Prediction time is: 8.35 ms
________________________________________________
*****************************************
File Name 9999963_00000_d_0000006
Cropped Image: 0

In [68]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp378/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.318
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 50 images (V1)**

In [69]:
result_predict_nms = predict(source='./test_subsets/v6',
                         dataset_json_path = './subset_visdrone_test_data_50_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/50 [00:00<?, ?it/s]

Image Name: 9999938_00000_d_0000283
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images:   2%|▏         | 1/50 [00:00<00:34,  1.43it/s]

Original Prediction Count 285
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  40
Final Bounding Box Count (NMS):  38
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  10
Prediction time is: 570.81 ms
Image Name: 9999938_00000_d_0000109
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


...

Performing inference on images:  98%|█████████▊| 49/50 [00:33<00:00,  1.82it/s]

Original Prediction Count 275
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  17
Prediction time is: 253.16 ms
Image Name: 9999973_00000_d_0000094
Image Size:  (1916, 1078)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images: 100%|██████████| 50/50 [00:34<00:00,  1.45it/s]

Original Prediction Count 69
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  6
Prediction time is: 906.79 ms


Prediction results are successfully exported to runs/predict/exp269
Model loaded in 0.031117916107177734 seconds.
Slicing performed in 0.07702803611755371 seconds.
Prediction performed in 28.148486614227295 seconds.
Exporting performed in 0.09485054016113281 seconds.


In [1]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './runs/predict/exp269/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.55s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.547
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
result_predict_nms = predict(source='./test_subsets/v6',
                         dataset_json_path = './subset_visdrone_test_data_50_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/50 [00:00<?, ?it/s]

Image Name: 9999938_00000_d_0000283
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images:   2%|▏         | 1/50 [00:08<07:10,  8.79s/it]

Original Prediction Count 285
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  40
Final Bounding Box Count (NMS):  38
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  10
Prediction time is: 8525.50 ms
Image Name: 9999938_00000_d_0000109
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


...

Performing inference on images:  98%|█████████▊| 49/50 [00:41<00:00,  1.79it/s]

Original Prediction Count 275
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  17
Prediction time is: 255.55 ms
Image Name: 9999973_00000_d_0000094
Image Size:  (1916, 1078)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images: 100%|██████████| 50/50 [00:43<00:00,  1.16it/s]

Original Prediction Count 69
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  6
Prediction time is: 918.88 ms


Prediction results are successfully exported to runs/predict/exp270
Model loaded in 0.03402590751647949 seconds.
Slicing performed in 0.18901467323303223 seconds.
Prediction performed in 36.65951633453369 seconds.
Exporting performed in 0.09498739242553711 seconds.


In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './runs/predict/exp270/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.54s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.547
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [24]:
#Union Merge
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 8.71 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 78.94 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.35 ms
Object Density: 3
Prediction time is: 7.35 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.36 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding 

In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp390/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.42s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.244
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.313
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.470
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 69.56 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 114.57 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.76 ms
Object Density: 3
Prediction time is: 7.76 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.65 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp379/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.50s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.313
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [17]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)



Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.58 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 81.15 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.45 ms
Object Density: 3
Prediction time is: 7.45 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.67 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding

In [18]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp380/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.47s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.244
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.313
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.470
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [19]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.79 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 112
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  45
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 199.07 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.46 ms
Object Density: 3
Prediction time is: 7.46 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.66 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxe

In [20]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp381/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.50s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.207
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.315
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.470
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [22]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.65 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  66
Final Bounding Box Count (OptNMS): 48
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 199.37 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.46 ms
Object Density: 3
Prediction time is: 7

In [23]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp382/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.122
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.126
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.317
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [24]:
result_predict_nms = predict(source='./test_subsets/v6',
                         dataset_json_path = './subset_visdrone_test_data_50_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/50 [00:00<?, ?it/s]

Image Name: 9999938_00000_d_0000283
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images:   2%|▏         | 1/50 [00:01<01:33,  1.91s/it]

Original Prediction Count 920
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  249
Final Bounding Box Count (NMS):  54
Final Bounding Box Count (NMS):  11
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  9
Prediction time is: 1739.27 ms
Image Name: 9999938_00000_d_0000109
Image Size:  (1400, 788)


...

Original Prediction Count 237
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  32
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  13
Prediction time is: 730.74 ms
Image Name: 9999973_00000_d_0000094
Image Size:  (1916, 1078)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images:  98%|█████████▊| 49/50 [01:36<00:01,  1.53s/it]

Original Prediction Count 490
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  212
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  16
Prediction time is: 2972.39 ms


Performing inference on images: 100%|██████████| 50/50 [01:36<00:00,  1.94s/it]

Prediction results are successfully exported to runs/predict/exp271
Model loaded in 0.04349970817565918 seconds.
Slicing performed in 0.2329418659210205 seconds.
Prediction performed in 90.15567350387573 seconds.
Exporting performed in 0.09753918647766113 seconds.


In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './runs/predict/exp271/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.88s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.091
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.057
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.331
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.595
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [26]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.88 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  56
Final Bounding Box Count (OptNMS): 55
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 539.99 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.54 ms
Object Density: 3
Prediction time is: 7.54 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.60 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  

In [27]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp383/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.53s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.241
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.329
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.470
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [29]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.81 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 70
Final Bounding Box Count (NMS):  51
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 535.77 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.39 ms
Object Density: 3
Prediction time is: 7.39 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.61 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 40
Final Bounding B

In [30]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp384/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.326
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.470
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.204
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [31]:
result_predict_nms = predict(source='./test_subsets/v6',
                         dataset_json_path = './subset_visdrone_test_data_50_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/50 [00:00<?, ?it/s]

Image Name: 9999938_00000_d_0000283
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images:   2%|▏         | 1/50 [00:06<05:07,  6.28s/it]

Original Prediction Count 500
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  181
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  26
Final Bounding Box Count (NMS):  1
Prediction time is: 6130.23 ms
Image Name: 9999938_00000_d_0000109
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


...

Original Prediction Count 226
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  70
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  9
Prediction time is: 2681.26 ms
Image Name: 9999973_00000_d_0000094
Image Size:  (1916, 1078)
Sliced Boxes Count: 1947
POST PROCESS:  NMS
Performing prediction on 1947 slices.


Performing inference on images:  98%|█████████▊| 49/50 [05:43<00:05,  5.36s/it]

Original Prediction Count 478
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  202
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  2
Prediction time is: 11479.54 ms


Performing inference on images: 100%|██████████| 50/50 [05:43<00:00,  6.87s/it]

Prediction results are successfully exported to runs/predict/exp272
Model loaded in 0.06510329246520996 seconds.
Slicing performed in 0.5836799144744873 seconds.
Prediction performed in 337.01868963241577 seconds.
Exporting performed in 0.09747433662414551 seconds.


In [32]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './runs/predict/exp272/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.85s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.112
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.113
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.381
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.587
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 50 images (V2)**

In [20]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v2/images'
json_path = "./subset_visdrone_test_data_50_v2.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000310
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.66 ms
Object Density: 5
Prediction time is: 10.66 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 8.22 ms
Object Density: 21
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 110
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  24
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 87.61 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.71 ms
Object Density: 1
Prediction time is: 7.71 ms
Cropped Image: 700 394 1400 788
Initial Prediction time is: 7.73 ms
Object Density: 7
Prediction time is: 7.73 ms
_

In [21]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './sliced_predictions/exp349/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.44s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.225
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.161
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [23]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v2/images'
json_path = "./subset_visdrone_test_data_50_v2.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000310
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.39 ms
Object Density: 5
Prediction time is: 10.39 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 8.65 ms
Object Density: 21
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  18
Final Bounding Box Count (OptNMS): 16
Adaptive Filtered Prediction:  49
Final Bounding Box Count (OptNMS): 34
Adaptive Filtered Prediction:  25
Final Bounding Box Count (OptNMS): 25
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 8

In [24]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './sliced_predictions/exp350/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.224
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [25]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v2/images'
json_path = "./subset_visdrone_test_data_50_v2.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000310
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.58 ms
Object Density: 5
Prediction time is: 10.58 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 8.19 ms
Object Density: 21
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 10
Adaptive Filtered Prediction:  39
Final Bounding Box Count (OptNMS): 36
Adaptive Filtered Prediction:  28
Final Bounding Box Count (OptNMS): 19
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 

In [26]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './sliced_predictions/exp351/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.214
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.360
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [28]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v2/images'
json_path = "./subset_visdrone_test_data_50_v2.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000310
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.78 ms
Object Density: 5
Prediction time is: 10.78 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 8.28 ms
Object Density: 21
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 89
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  19
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 206.77 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.74 ms
Object Density: 1
Prediction time is: 7.74 ms
Cropped Image: 700 394 1400 788
Initial Prediction time is: 7.81 ms
Object Density: 7
Prediction time is: 7.81 ms


In [29]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './sliced_predictions/exp352/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.49s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.214
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.358
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.227
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [32]:
result_predict_nms = predict(source='./test_subsets/v2',
                         dataset_json_path = './subset_visdrone_test_data_50_v2.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 128,
                         slice_width = 128,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/50 [00:00<?, ?it/s]

Image Name: 9999938_00000_d_0000310
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


Performing inference on images:   2%|▏         | 1/50 [00:01<01:33,  1.91s/it]

Original Prediction Count 389
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  81
Final Bounding Box Count (NMS):  48
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  3
Prediction time is: 1772.31 ms
Image Name: 9999938_00000_d_0000019
Image Size:  (1400, 788)
Sliced Boxes Count: 252
POST PROCESS:  NMS
Performing prediction on 252 slices.


...

Original Prediction Count 183
Final Bounding Box Count (NMS):  35
Final Bounding Box Count (NMS):  31
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  3
Prediction time is: 1654.95 ms
Image Name: 9999973_00000_d_0000094
Image Size:  (1916, 1078)
Sliced Boxes Count: 464
POST PROCESS:  NMS
Performing prediction on 464 slices.


Performing inference on images:  98%|█████████▊| 49/50 [01:39<00:01,  1.80s/it]

Original Prediction Count 490
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  212
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  16
Prediction time is: 3080.70 ms


Performing inference on images: 100%|██████████| 50/50 [01:39<00:00,  1.99s/it]

Prediction results are successfully exported to runs/predict/exp258
Model loaded in 0.038781166076660156 seconds.
Slicing performed in 0.22474884986877441 seconds.
Prediction performed in 92.76929473876953 seconds.
Exporting performed in 0.09723663330078125 seconds.


In [33]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './runs/predict/exp258/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.90s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.094
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.324
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.576
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [41]:
result_predict_nms = predict(source='./test_subsets/v2',
                         dataset_json_path = './subset_visdrone_test_data_50_v2.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 64,
                         slice_width = 64,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/50 [00:00<?, ?it/s]

Image Name: 9999938_00000_d_0000310
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


Performing inference on images:   2%|▏         | 1/50 [00:06<05:10,  6.34s/it]

Original Prediction Count 80
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  36
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  6
Prediction time is: 6224.65 ms
Image Name: 9999938_00000_d_0000019
Image Size:  (1400, 788)
Sliced Boxes Count: 1032
POST PROCESS:  NMS
Performing prediction on 1032 slices.


...

Original Prediction Count 98
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  4
Prediction time is: 6113.18 ms
Image Name: 9999973_00000_d_0000094
Image Size:  (1916, 1078)
Sliced Boxes Count: 1947
POST PROCESS:  NMS
Performing prediction on 1947 slices.


Performing inference on images:  98%|█████████▊| 49/50 [05:48<00:06,  6.33s/it]

Original Prediction Count 478
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  202
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  2
Prediction time is: 11789.01 ms


Performing inference on images: 100%|██████████| 50/50 [05:49<00:00,  6.98s/it]

Prediction results are successfully exported to runs/predict/exp259
Model loaded in 0.029069900512695312 seconds.
Slicing performed in 0.6582772731781006 seconds.
Prediction performed in 342.77222084999084 seconds.
Exporting performed in 0.09673762321472168 seconds.


In [42]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './runs/predict/exp259/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.74s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.167
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.372
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.584
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.019
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.230
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [43]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v2/images'
json_path = "./subset_visdrone_test_data_50_v2.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000310
Cropped Image: 0 0 700 394
Initial Prediction time is: 71.59 ms
Object Density: 5
Prediction time is: 71.59 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.98 ms
Object Density: 21
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 30
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  21
Sliced Prediction time is: 549.03 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.62 ms
Object Density: 1
Prediction time is: 7.62 ms
Cropped Image: 700 394 1400 788
Initial Prediction time is: 7.69 ms
Object Density: 7
Prediction time is: 7.69 ms
________________________________________________
*****************************************
File Name 9999938_00000_d_0000019
Cropped Image: 0 0 

In [44]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './sliced_predictions/exp353/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.45s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.237
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.150
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.241
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [46]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v2/images'
json_path = "./subset_visdrone_test_data_50_v2.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000310
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.79 ms
Object Density: 5
Prediction time is: 10.79 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 8.23 ms
Object Density: 21
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  22
Final Bounding Box Count (OptNMS): 22
Sliced Prediction time is: 550.53 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.67 ms
Object Density: 1
Prediction time is: 7.67 ms
Cropped Image: 700 394 1400 788
Initial Prediction time is: 7.71 ms
Object Density: 7
Prediction time is: 7.71 ms
________________________________________________
**

In [47]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v2.json' --result_json_path './sliced_predictions/exp354/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.236
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.241
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [ ]:
#small area --> low IoU threshold
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v2/images'
json_path = "./subset_visdrone_test_data_50_v2.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)

In [ ]:
result_predict_nms = predict(source='./test_subsets/v1',
                         dataset_json_path = './subset_visdrone_test_data_15_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

In [ ]:
result_predict_nms = predict(source='./test_subsets/v1',
                         dataset_json_path = './subset_visdrone_test_data_15_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

In [ ]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v1.json' --result_json_path './sliced_predictions/exp346/result.json'